# Import libraries

In [58]:
import pandas as pd
from pathlib import Path

# Set path

In [ ]:
yearmonth = "202512"

In [60]:
base = Path("B:/02_Grade_Control/01_Blasthole Dat/01_grade_control").resolve()

source_dir = base / '03_grade_samples'
output_dir = base / rf"04_database/{yearmonth[:4]}/{yearmonth}"

# Retrieve CSV files

In [61]:
gs_dir = source_dir / yearmonth[:4] / yearmonth
grade_samples = list(gs_dir.rglob("*_db.csv"))
print(len(grade_samples), 'CSV files')
grade_samples

13 CSV files


[WindowsPath('//172.16.2.10/SharedFolder/Geology/H Folder/02_Grade_Control/01_Blasthole Dat/01_grade_control/03_grade_samples/2025/202511/20251107/bh_519_63_hill_2_345_t_20251107_db.csv'),
 WindowsPath('//172.16.2.10/SharedFolder/Geology/H Folder/02_Grade_Control/01_Blasthole Dat/01_grade_control/03_grade_samples/2025/202511/20251109/bh_519_58_hill_2_345_t_20251109_db.csv'),
 WindowsPath('//172.16.2.10/SharedFolder/Geology/H Folder/02_Grade_Control/01_Blasthole Dat/01_grade_control/03_grade_samples/2025/202511/20251110/bh_35b_lapyahan_225_v1_20251110_db.csv'),
 WindowsPath('//172.16.2.10/SharedFolder/Geology/H Folder/02_Grade_Control/01_Blasthole Dat/01_grade_control/03_grade_samples/2025/202511/20251111/bh_519_31_32b_lusong_225_t_rev1_20251111_db.csv'),
 WindowsPath('//172.16.2.10/SharedFolder/Geology/H Folder/02_Grade_Control/01_Blasthole Dat/01_grade_control/03_grade_samples/2025/202511/20251112/bh_337_36_lapyahan_225_20251112_db.csv'),
 WindowsPath('//172.16.2.10/SharedFolder/Geolo

In [62]:
csv_files = []

for item in grade_samples:
    df = pd.read_csv(item)
    csv_files.append(df)

# Merged data

In [63]:
df_merged = pd.concat(csv_files)

df_merged["max_depth"] = df_merged["z_lh"] - df_merged["z_dp"]
df_merged['depth_from'] = 0

df_merged

,string_lh,y_lh,x_lh,z_lh,Cu_final,hole_id,d3,d4,BHID,Drill Plan,z_dp,Unique ID,Sample ID,CuO_Soluble,Au,Ag,max_depth,depth_from
0,1,15783.542,30878.930,363.455,0.08,134,RTK,20251107,20251107_345N_134,63_Hill_2_345,343.7,COP25-17878,134,NaN,NaN,NaN,19.755,0
1,1,15770.425,30886.839,362.607,0.06,128,RTK,20251107,20251107_345N_128,63_Hill_2_345,345.0,COP25-17879,128,0.001584,NaN,NaN,17.607,0
2,1,15773.939,30891.805,362.754,0.08,124,RTK,20251107,20251107_345N_124,63_Hill_2_345,344.0,COP25-17880,124,NaN,NaN,NaN,18.754,0
3,1,15829.926,30894.475,361.852,0.11,75,RTK,20251107,20251107_345N_75,63_Hill_2_345,345.0,COP25-17881,75,NaN,NaN,NaN,16.852,0
4,1,15773.240,30876.987,362.582,0.08,133,RTK,20251107,20251107_345N_133,63_Hill_2_345,343.7,COP25-17883,133,0.001642,NaN,NaN,18.882,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,1,15139.289,30324.614,238.705,0.08,35,RTK,20251130,20251130_225S_35,25_Midlusong_225,225.0,COP25-19168,35,NaN,NaN,NaN,13.705,0
49,1,15134.048,30325.577,238.904,0.05,22,RTK,20251130,20251130_225S_22,25_Midlusong_225,224.0,COP25-19169,22,0.0029,NaN,NaN,14.904,0
50,1,15148.093,30337.448,239.076,0.05,18,RTK,20251130,20251130_225S_18,25_Midlusong_225,224.0,COP25-19170,18,NaN,NaN,NaN,15.076,0
51,1,15122.875,30317.285,239.285,0.06,25,RTK,20251130,20251130_225S_25,25_Midlusong_225,224.0,COP25-19171,25,NaN,NaN,NaN,15.285,0


# Convert to database format

## Collar

In [64]:
df_collar = df_merged.copy()
df_collar = df_collar[["BHID", "x_lh", "y_lh", "z_lh", "max_depth", 'd4']]

df_collar.columns = ['hole_id', 'x', 'y', 'z', 'max_depth', 'date']

df_collar["date"] = df_collar["date"].astype(str)
df_collar['date'] = pd.to_datetime(df_collar['date'], format='%Y%m%d', errors='coerce')
df_collar["date"] = df_collar["date"].dt.strftime("%Y-%m-%d")
df_collar["max_depth"] = df_collar["max_depth"].round(3)

df_collar

,hole_id,x,y,z,max_depth,date
0,20251107_345N_134,30878.930,15783.542,363.455,19.755,2025-11-07
1,20251107_345N_128,30886.839,15770.425,362.607,17.607,2025-11-07
2,20251107_345N_124,30891.805,15773.939,362.754,18.754,2025-11-07
3,20251107_345N_75,30894.475,15829.926,361.852,16.852,2025-11-07
4,20251107_345N_133,30876.987,15773.240,362.582,18.882,2025-11-07
...,...,...,...,...,...,...
48,20251130_225S_35,30324.614,15139.289,238.705,13.705,2025-11-30
49,20251130_225S_22,30325.577,15134.048,238.904,14.904,2025-11-30
50,20251130_225S_18,30337.448,15148.093,239.076,15.076,2025-11-30
51,20251130_225S_25,30317.285,15122.875,239.285,15.285,2025-11-30


## Survey

In [65]:
df_survey = df_merged.copy()
df_survey = df_survey[['BHID']]

df_survey['azimuth'] = 0
df_survey['depth'] = 0
df_survey['dip'] = -90

df_survey

,BHID,azimuth,depth,dip
0,20251107_345N_134,0,0,-90
1,20251107_345N_128,0,0,-90
2,20251107_345N_124,0,0,-90
3,20251107_345N_75,0,0,-90
4,20251107_345N_133,0,0,-90
...,...,...,...,...
48,20251130_225S_35,0,0,-90
49,20251130_225S_22,0,0,-90
50,20251130_225S_18,0,0,-90
51,20251130_225S_25,0,0,-90


## Assay

In [66]:
df_assay = df_merged.copy()
df_assay = df_assay[['BHID','Unique ID', 'depth_from', 'max_depth', "Cu_final", "Au", "Ag", "CuO_Soluble"]]

# Ensure numeric + apply precision
df_assay["Cu_final"] = pd.to_numeric(df_assay["Cu_final"], errors="coerce").round(4)
df_assay["CuO_Soluble"] = pd.to_numeric(df_assay["CuO_Soluble"], errors="coerce").round(4)
df_assay["max_depth"] = pd.to_numeric(df_assay["max_depth"], errors="coerce").round(3)

df_assay

,BHID,Unique ID,depth_from,max_depth,Cu_final,Au,Ag,CuO_Soluble
0,20251107_345N_134,COP25-17878,0,19.755,0.08,NaN,NaN,NaN
1,20251107_345N_128,COP25-17879,0,17.607,0.06,NaN,NaN,0.0016
2,20251107_345N_124,COP25-17880,0,18.754,0.08,NaN,NaN,NaN
3,20251107_345N_75,COP25-17881,0,16.852,0.11,NaN,NaN,NaN
4,20251107_345N_133,COP25-17883,0,18.882,0.08,NaN,NaN,0.0016
...,...,...,...,...,...,...,...,...
48,20251130_225S_35,COP25-19168,0,13.705,0.08,NaN,NaN,NaN
49,20251130_225S_22,COP25-19169,0,14.904,0.05,NaN,NaN,0.0029
50,20251130_225S_18,COP25-19170,0,15.076,0.05,NaN,NaN,NaN
51,20251130_225S_25,COP25-19171,0,15.285,0.06,NaN,NaN,NaN


# Export CSVs

In [67]:
output_dir.mkdir(parents=True, exist_ok=True)

In [68]:
df_collar.to_csv(output_dir / "collar.csv", index=False)
df_survey.to_csv(output_dir / "survey.csv", index=False)
df_assay.to_csv(output_dir / "assay.csv", index=False)